### Etapa 01

- División y Preprocesamiento de Videos

    - Cortar el video en segmentos (ej. cada 10 segundos, 30 segundos o 1 minuto) para procesar por lotes.

#### Instalación de dependencias necesarias :

- *pip install opencv-python deepface mediapipe pandas matplotlib*

In [6]:
import cv2
import os
from datetime import datetime, timedelta
from typing import Optional


#### OpenCv para la extracción de frames

In [7]:
def extract_frames(video_path: str, output_folder: str, interval_sec: int = 10) -> None:
    """
    Extrae frames de un video cada `interval_sec` segundos y los guarda como imágenes.
    Args:
        video_path (str): Ruta del archivo de video.
        output_folder (str): Carpeta donde se guardarán los frames extraídos.
        interval_sec (int): Intervalo de tiempo en segundos entre cada frame guardado.
    """
    # Extraer timestamp inicial desde el nombre del archivo
    video_name: str = os.path.basename(video_path).split('.')[0]
    try:
        start_time: datetime = datetime.strptime(video_name, "%Y-%m-%d_%H-%M-%S")
    except ValueError:
        print("Error: El nombre del archivo debe tener el formato 'YYYY-MM-DD_HH-MM-SS.ext'")
        return


    # Eliminar la carpeta si existe
    if os.path.exists(output_folder):
        print(f"❌ La carpeta '{output_folder}' ya existe. Eliminando...")
        for file in os.listdir(output_folder):
            file_path: str = os.path.join(output_folder, file)
            # print(f"Eliminando {file_path}...")
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    os.rmdir(file_path)
            except Exception as e:
                print(f"Error al eliminar {file_path}: {e}")
    # Crear la carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)

    # Abrir el video con OpenCV
    cap: cv2.VideoCapture = cv2.VideoCapture(video_path)

    # Obtener los frames por segundo del video (para calcular intervalos por tiempo)
    fps: float = cap.get(cv2.CAP_PROP_FPS)
    print(f"FPS del video: {fps}")

    # Convertir segundos a cantidad de frames (por ejemplo: 10 seg → 300 frames si fps=30)
    frame_interval: int = int(fps * interval_sec)
    print(f"Intervalo de frames: {frame_interval} frames ({interval_sec} segundos)")

    # Total de frames en el video (para saber cuándo detenerse)
    total_frames: int = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total de frames en el video: {total_frames}")

    count: int = 0       # Contador de frames leídos
    saved: int = 0       # Contador de frames guardados

    while cap.isOpened():
        ret: bool
        frame: cv2.typing.MatLike
        ret, frame = cap.read()

        # Si no se pudo leer un frame (fin del video), salir del bucle
        if not ret:
            break

        # Guardar el frame si corresponde al intervalo especificado
        if count % frame_interval == 0:
            timestamp_sec: float = count / fps
            current_time: datetime = start_time + timedelta(seconds=timestamp_sec)
            filename: str = current_time.strftime("%Y-%m-%d_%H-%M-%S") + ".jpg"
            output_path: str = os.path.join(output_folder, filename)

            cv2.imwrite(output_path, frame)
            saved += 1
        count += 1

    # Liberar el recurso del video
    cap.release()
    print(f"{saved} frames extraídos correctamente en '{output_folder}'.")

# 🧪 Ejemplo de uso:
extract_frames("2025-04-25_12-26-31.mkv", "frames_output", interval_sec=5)


❌ La carpeta 'frames_output' ya existe. Eliminando...
FPS del video: 60.0
Intervalo de frames: 300 frames (5 segundos)
Total de frames en el video: 6292
21 frames extraídos correctamente en 'frames_output'.
